In [2]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
import json
import sys 
import warnings
warnings.filterwarnings('ignore')
import gradio as gr
import torch
import json
import IPython.display as ipd
import faiss
import soundfile as sf
import fitz  
import re
import gradio as gr
import time
from datetime import datetime
import seaborn as sns


from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from IPython.display import Audio
from sentence_transformers import SentenceTransformer
from openai import OpenAI
from nemo.collections.tts.models import FastPitchModel
from pathlib import Path
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.models import FastPitchModel
from scipy.io.wavfile import write
from transformers import pipeline
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score



2025-12-03 21:20:18.359272: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-03 21:20:18.407101: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-03 21:20:18.407133: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-03 21:20:18.407158: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-03 21:20:18.414377: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

## estimate gender

In [3]:
def estimate_gender(audio_path):
    
    y, sr = librosa.load(audio_path, sr=None)
    
    f0 = librosa.pyin(y, fmin=75, fmax=300, sr=sr, 
                     frame_length=2048, win_length=1024, hop_length=256)
    f0_values = f0[0]
    f0_values = f0_values[~np.isnan(f0_values)]
    
    if len(f0_values) == 0:
        return "Unknown"
    
    f0_median = np.median(f0_values)
    print(f0_median)
   
    if f0_median <= 130:
        return "Male"
    elif f0_median >= 165:
        return "Female"
    else:
       
        f0_mean = np.mean(f0_values)
        f0_std = np.std(f0_values)
        
        if f0_median < 145 and f0_std < 25:
            return "Male"
        else:
            return "Female"
        


## LSTM emotion rec.


### Előkészítés

In [4]:
val_paths = []
val_labels = []
for i in range(19,21):
    for dirname, _, filenames in os.walk(f"/home/simon/ESD/00{i}/Surprise"):
        for filename in filenames:
            val_paths.append(os.path.join(dirname,filename))
            label="surprise"
            val_labels.append(label.lower())

print("Dataset is Loaded")
for i in range(19,21):
    for dirname, _, filenames in os.walk(f"/home/simon/ESD/00{i}/Sad"):
        for filename in filenames:
            val_paths.append(os.path.join(dirname,filename))
            label="sad"
            val_labels.append(label.lower())

print("Dataset is Loaded")
for i in range(19,21):
    for dirname, _, filenames in os.walk(f"/home/simon/ESD/00{i}/Happy"):
        for filename in filenames:
            val_paths.append(os.path.join(dirname,filename))
            label="happy"
            val_labels.append(label.lower())

print("Dataset is Loaded")
for i in range(19,21):
    for dirname, _, filenames in os.walk(f"/home/simon/ESD/00{i}/Angry"):
        for filename in filenames:
            val_paths.append(os.path.join(dirname,filename))
            label="angry"
            val_labels.append(label.lower())

print("Dataset is Loaded")
for i in range(19,21):
    for dirname, _, filenames in os.walk(f"/home/simon/ESD/00{i}/Neutral"):
        for filename in filenames:
            val_paths.append(os.path.join(dirname,filename))
            label="neutral"
            val_labels.append(label.lower())

print("Dataset is Loaded")
df_val= pd.DataFrame()
df_val['speech']=val_paths
df_val['label']=val_labels
df_val.head()

Dataset is Loaded
Dataset is Loaded
Dataset is Loaded
Dataset is Loaded
Dataset is Loaded


,speech,label
0,/home/simon/ESD/0019/Surprise/0019_001592.wav,surprise
1,/home/simon/ESD/0019/Surprise/0019_001572.wav,surprise
2,/home/simon/ESD/0019/Surprise/0019_001613.wav,surprise
3,/home/simon/ESD/0019/Surprise/0019_001741.wav,surprise
4,/home/simon/ESD/0019/Surprise/0019_001512.wav,surprise


In [5]:
def extract_mfcc(filename):
    y,sr=librosa.load(filename, duration=3, offset=0.5)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return mfcc

In [6]:
extract_mfcc(df_val['speech'][0])

array([-3.5702924e+02,  9.6109161e+01,  1.8062576e+00,  2.2582365e+01,
       -3.1091631e+01, -1.7898973e+01, -1.5407653e+01, -4.4629386e-01,
       -2.4464369e+01, -7.5679815e-01, -7.0336785e+00, -1.7772552e+01,
        1.0111096e+00, -1.0597541e+01, -1.2515818e+00, -6.2318811e+00,
        2.3008943e+00, -1.0849600e+00, -3.3185184e+00,  8.5988939e-01,
       -1.7053005e+00, -4.2913432e+00, -3.9740238e+00, -1.0241350e+00,
       -4.7427807e+00, -3.0093232e-01, -1.0000892e+00,  4.3346378e-01,
       -5.7896429e-01,  9.5399719e-01, -4.0370992e-01, -2.1399801e+00,
       -1.8853335e+00,  9.6497232e-01,  2.8253663e+00,  1.4093702e+00,
        5.7589135e+00,  3.7085714e+00,  5.6110139e+00,  4.2233911e+00],
      dtype=float32)

In [7]:
X_mfcc_val = df_val['speech'].apply(lambda x: extract_mfcc(x))
X_mfcc_val
X_val_esd = [x for x in X_mfcc_val]
X_val_esd = np.array(X_val_esd)
X_val_esd.shape
X_val_esd = np.expand_dims(X_val_esd,-1)
X_val_esd.shape
X_val_esd=X_val_esd.astype('float32')
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
y_val_esd = enc.fit_transform(df_val[['label']]).astype('float32')

print(y_val_esd.shape)  
print(np.unique(y_val_esd)) 

(3500, 5)
[0. 1.]


In [8]:
paths = []
labels = []

new_dataset_path = "/root/.cache/kagglehub/datasets/dejolilandry/asvpesdspeech-nonspeech-emotional-utterances/versions/19/ASVP-ESD-Update/Audio"

allowed_emotions = {
    "02": "neutral",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "08": "surprise"
}

for root, dirs, files in os.walk(new_dataset_path):
    for file in files:
        if not file.endswith(".wav"):
            continue
        
        parts = file.split("-")
        if len(parts) < 9:
            continue 
        
        emotion_code = parts[2]
        language_code = parts[8]
        
        if emotion_code in allowed_emotions and language_code=="02":
            paths.append(os.path.join(root, file))
            labels.append(allowed_emotions[emotion_code])


for i in range(11,18):
    for dirname, _, filenames in os.walk(f"/home/simon/ESD/00{i}/Happy"):
        for filename in filenames:
            paths.append(os.path.join(dirname, filename))
            labels.append("happy")

for i in range(11,18):
    for dirname, _, filenames in os.walk(f"/home/simon/ESD/00{i}/Angry"):
        for filename in filenames:
            paths.append(os.path.join(dirname, filename))
            labels.append("angry")

for i in range(11,18):
    for dirname, _, filenames in os.walk(f"/home/simon/ESD/00{i}/Neutral"):
        for filename in filenames:
            paths.append(os.path.join(dirname, filename))
            labels.append("neutral")

for i in range(11,18):
    for dirname, _, filenames in os.walk(f"/home/simon/ESD/00{i}/Surprise"):
        for filename in filenames:
            paths.append(os.path.join(dirname, filename))
            labels.append("surprise")

for i in range(11,18):
    for dirname, _, filenames in os.walk(f"/home/simon/ESD/00{i}/Sad"):
        for filename in filenames:
            paths.append(os.path.join(dirname, filename))
            labels.append("sad")

print("Total samples:", len(paths))


Total samples: 14619


In [9]:
df= pd.DataFrame()
df['speech']=paths
df['label']=labels
df.head()

,speech,label
0,/root/.cache/kagglehub/datasets/dejolilandry/a...,angry
1,/root/.cache/kagglehub/datasets/dejolilandry/a...,surprise
2,/root/.cache/kagglehub/datasets/dejolilandry/a...,surprise
3,/root/.cache/kagglehub/datasets/dejolilandry/a...,surprise
4,/root/.cache/kagglehub/datasets/dejolilandry/a...,surprise


In [10]:
extract_mfcc(df['speech'][0])

array([-600.97076  ,  100.06333  ,  -42.26156  ,   35.409367 ,
         16.721735 ,   15.365932 ,  -14.453442 ,    3.399201 ,
         -1.9597461,  -25.75174  ,   -6.6531873,   12.56852  ,
          7.4000473,    6.1022797,   -7.207781 ,  -19.00995  ,
         -9.826457 ,    3.327453 ,    1.8901261,   -8.150221 ,
         -1.2986673,   -3.23105  ,    5.3001704,    2.9816468,
         -3.1129587,   -4.3711624,  -14.622587 ,   -1.1081332,
         -2.4216607,   -8.203386 ,   -5.3483057,   -7.2141714,
         -2.3185892,    5.3036013,   10.792786 ,    8.713029 ,
          3.4217293,   -2.552914 ,   -5.5329266,   -2.4955325],
      dtype=float32)

In [75]:
import os

bad_paths = [p for p in df['speech'] if not os.path.isfile(p)]
print("Bad path num:", len(bad_paths))
print(bad_paths[:10])

Bad path num: 0
[]


In [12]:
bad_path = "/home/simon/ESD/0013/Sad/Sad"
df = df[df['speech'] != bad_path]


In [13]:
X_mfcc = df['speech'].apply(lambda x: extract_mfcc(x))

In [14]:
X = [x for x in X_mfcc]
X = np.array(X)
X.shape

(14618, 40)

In [15]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
y = enc.fit_transform(df[['label']]).astype('float32')

print(y.shape)
print(np.unique(y)) 

(14618, 5)
[0. 1.]


### LSTM

In [16]:
import numpy as np
import tensorflow as tf
import pandas as pd
import scipy

from keras.models import Sequential
from keras.layers import Dense, LSTM , Dropout
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model = Sequential([
    LSTM(123,return_sequences=False, input_shape=(40,1)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(5, activation='softmax')
])

model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

2025-12-03 21:24:57.495551: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 123)               61500     
                                                                 
 dense (Dense)               (None, 64)                7936      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 165       
                                                                 
Total params: 71681 (280.00 KB)
Trainable params: 71681 

In [17]:
y_onehot = to_categorical(y, num_classes=5)

In [18]:
indices_train = np.arange(X.shape[0])
np.random.shuffle(indices_train)
X = X[indices_train]
y = y[indices_train]

indices_val = np.arange(X_val_esd.shape[0])
np.random.shuffle(indices_val)
X_val_esd = X_val_esd[indices_val]
y_val_esd = y_val_esd[indices_val]

history = model.fit(X, y,
                    validation_data=(X_val_esd, y_val_esd),
                    epochs=100,
                    batch_size=512,
                    shuffle=True) 


Epoch 1/100
29/29 [==============================] - 6s 110ms/step - loss: 1.5628 - accuracy: 0.2831 - val_loss: 1.5330 - val_accuracy: 0.2860
Epoch 2/100
29/29 [==============================] - 3s 86ms/step - loss: 1.4569 - accuracy: 0.3578 - val_loss: 1.5471 - val_accuracy: 0.3000
Epoch 3/100
29/29 [==============================] - 3s 87ms/step - loss: 1.3764 - accuracy: 0.3981 - val_loss: 1.4581 - val_accuracy: 0.3477
Epoch 4/100
29/29 [==============================] - 2s 83ms/step - loss: 1.2954 - accuracy: 0.4360 - val_loss: 1.4009 - val_accuracy: 0.3814
Epoch 5/100
29/29 [==============================] - 2s 81ms/step - loss: 1.2571 - accuracy: 0.4533 - val_loss: 1.3899 - val_accuracy: 0.3877
Epoch 6/100
29/29 [==============================] - 2s 82ms/step - loss: 1.2298 - accuracy: 0.4703 - val_loss: 1.5459 - val_accuracy: 0.3286
Epoch 7/100
29/29 [==============================] - 2s 82ms/step - loss: 1.1977 - accuracy: 0.4843 - val_loss: 1.5992 - val_accuracy: 0.3446
Epoch

In [19]:
y_pred_esd = model.predict(X_val_esd) 
pred_classes_esd = np.argmax(y_pred_esd, axis=1)
true_classes_esd = np.argmax(y_val_esd, axis=1)

accuracy = accuracy_score(true_classes_esd, pred_classes_esd)
print(f"Validation Accuracy: {accuracy*100:.2f}%")

print(classification_report(true_classes_esd, pred_classes_esd))


110/110 [==============================] - 2s 11ms/step
Validation Accuracy: 32.66%
              precision    recall  f1-score   support

           0       0.34      0.37      0.35       700
           1       0.26      0.30      0.28       700
           2       0.35      0.52      0.42       700
           3       0.35      0.19      0.24       700
           4       0.36      0.26      0.30       700

    accuracy                           0.33      3500
   macro avg       0.33      0.33      0.32      3500
weighted avg       0.33      0.33      0.32      3500



### Prediction

In [20]:
def decode_onehot(pred_class, encoder):
    onehot = np.zeros((1, len(encoder.categories_[0])))  
    onehot[0, pred_class] = 1
    return encoder.inverse_transform(onehot)[0][0]


In [21]:
def predict_emotion(audio_path, model, encoder, sr=22050, n_mfcc=40):

    y, sr = librosa.load(audio_path, sr=sr)

    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc.T, axis=0)

    mfcc_input = mfcc_mean.reshape(1, n_mfcc, 1) 

    pred = model.predict(mfcc_input)
    pred_class = np.argmax(pred, axis=1)[0]

    return decode_onehot(pred_class, encoder)


In [22]:
audio_path="/home/simon/ESD/0019/Happy/0019_000707.wav"
predicted_emotion = predict_emotion(audio_path, model, enc)
print("Predicted emotion:", predicted_emotion)


1/1 [==============================] - 0s 17ms/step
Predicted emotion: surprise


## whisper 

In [23]:

whisper = pipeline(
    task="automatic-speech-recognition",
    model="openai/whisper-medium",
    device=0 if torch.cuda.is_available() else -1,
    generate_kwargs={
        "language": "en",       
        "task": "transcribe"   
    }
)


def hang_felismeres(hang):
    if hang is None:
        return None, "Nem kaptam hangot."

    samplerate, data = hang  
    filepath = "felvetel.wav"


    write(filepath, samplerate, data.astype(np.int16))

    eredmeny = whisper(filepath)
    text = eredmeny["text"]

    return filepath, text




## Rag

In [24]:

def load_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text


In [25]:


def smart_chunk_text(text, max_chunk_size=800, overlap=120):
    chunks = []

    title_pattern = re.compile(r"(^|\n)([A-Z][A-Za-z0-9 ,:-]{3,})\n")
    parts = title_pattern.split(text)

    paired_sections = []
    current_title = "Unknown section"
    
    for i in range(len(parts)):
        part = parts[i].strip()
        if i % 3 == 2:   
            current_title = part
        elif i % 3 == 0 and part:
            paired_sections.append((current_title, part))
    
    for title, content in paired_sections:
        paragraphs = [p.strip() for p in content.split("\n\n") if len(p.strip()) > 0]
        current_chunk = ""

        for para in paragraphs:
            if len(current_chunk) + len(para) < max_chunk_size:
                current_chunk += para + "\n\n"
            else:
                chunks.append({"title": title, "content": current_chunk.strip()})
                overlap_text = current_chunk[-overlap:]
                current_chunk = overlap_text + para + "\n\n"

        if current_chunk.strip():
            chunks.append({"title": title, "content": current_chunk.strip()})

    return chunks


In [26]:

embedder = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

def create_embeddings(chunks):
    text_data = [ch["title"] + "\n" + ch["content"] for ch in chunks]
    
    embeddings = embedder.encode(text_data, normalize_embeddings=True)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))
    
    return index, embeddings, text_data


In [27]:
def retrieve_context(query, index, text_data, top_k=3):
    query_emb = embedder.encode([query], normalize_embeddings=True)
    distances, indices = index.search(np.array(query_emb), top_k)
    results = [text_data[i] for i in indices[0]]
    return "\n\n---\n\n".join(results)


## Fastpitch modell

In [28]:
home_path = !(echo $HOME)
home_path = home_path[0]
print(home_path)

/root


In [29]:
os.chdir("..")
!pwd

/home/simon


In [30]:
model_path = Path("./fastpitch_10speaker.nemo")
pretrained_model = FastPitchModel.restore_from(model_path)

 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
I1203 21:29:08.430548 140008601651008 tokenize_and_classify.py:87] Creating ClassifyFst grammars.
[NeMo W 2025-12-03 21:29:33 en_us_arpabet:66] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2025-12-03 21:29:33 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/HiFiTTS/manifest_train_ngc.json
      sample_rate: 44100
      sup_data_path: /raid/sup_data_path_44100_new
      sup_data_types:
      - align_prior_matrix
      - pitch
      - speaker

[NeMo I 2025-12-03 21:29:33 features:289] PADDING: 1
[NeMo I 2025-12-03 21:29:34 save_restore_connector:249] Model FastPitchModel was successfully restored from /home/simon/fastpitch_10speaker.nemo.


In [31]:
vocoder = HifiGanModel.from_pretrained("tts_en_hifitts_hifigan_ft_fastpitch")
vocoder = vocoder.eval().cuda()

[NeMo I 2025-12-03 21:29:34 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.23.0/tts_en_hifitts_hifigan_ft_fastpitch/dcbdefa79d5e3587c93636d4f53260b8/tts_en_hifitts_hifigan_ft_fastpitch.nemo.
[NeMo I 2025-12-03 21:29:34 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.23.0/tts_en_hifitts_hifigan_ft_fastpitch/dcbdefa79d5e3587c93636d4f53260b8/tts_en_hifitts_hifigan_ft_fastpitch.nemo
[NeMo I 2025-12-03 21:29:34 common:924] Instantiating model from pre-trained checkpoint


[NeMo W 2025-12-03 21:29:34 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.VocoderDataset
      manifest_filepath: /manifests/hifitts_fp_1000epoch_mels_train.json
      sample_rate: 44100
      n_segments: 16384
      max_duration: null
      min_duration: 0.75
      load_precomputed_mel: true
      hop_length: 512
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 100
      num_workers: 4
    
[NeMo W 2025-12-03 21:29:34 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.VocoderDataset


[NeMo I 2025-12-03 21:29:34 features:289] PADDING: 0
[NeMo I 2025-12-03 21:29:34 features:297] STFT using exact pad
[NeMo I 2025-12-03 21:29:34 features:289] PADDING: 0
[NeMo I 2025-12-03 21:29:34 features:297] STFT using exact pad
[NeMo I 2025-12-03 21:29:35 save_restore_connector:249] Model HifiGanModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.23.0/tts_en_hifitts_hifigan_ft_fastpitch/dcbdefa79d5e3587c93636d4f53260b8/tts_en_hifitts_hifigan_ft_fastpitch.nemo.


In [32]:


def infer_multispeaker(spec_gen_model, vocoder_model, text, speaker_id=0, pace=1.0):
    
    spec_gen_model.eval()
    vocoder_model.eval()

    with torch.no_grad():
      
        tokens = spec_gen_model.parse(text)
        if tokens.ndim == 1:
            tokens = tokens.unsqueeze(0)

        device = next(spec_gen_model.parameters()).device
        tokens = tokens.to(device)

      
        n_speakers = getattr(spec_gen_model, "n_speakers", 1) 
        if n_speakers > 1:
            if speaker_id is None or speaker_id >= n_speakers:
                raise ValueError(f"speaker_id must be < {n_speakers}, got {speaker_id}")
            speaker = torch.tensor([speaker_id], device=device, dtype=torch.long)
        else:
            speaker = None
            
        spectrogram = spec_gen_model.generate_spectrogram(
            tokens=tokens,
            speaker=speaker,
            pace=pace,
        )

        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)

    if isinstance(spectrogram, torch.Tensor):
        spectrogram = spectrogram.squeeze().cpu().numpy()
    if isinstance(audio, torch.Tensor):
        audio = audio.squeeze().cpu().numpy()

    return spectrogram, audio


In [33]:


def get_last_ckpt(exp_dir, model_name):
 
    base = Path(exp_dir) / model_name
    if not base.exists():
        raise FileNotFoundError(f"Nincs ilyen mappa: {base}")

    runs = sorted(
    [d for d in base.iterdir() if d.is_dir() and ".ipynb_checkpoints" not in str(d)],
    key=lambda x: x.stat().st_mtime
)

    if not runs:
        raise FileNotFoundError(f"Nincs kísérlet a {base} mappában.")

    last_run = runs[-1]
    ckpt_dir = last_run / "checkpoints"

    last_ckpts = sorted(ckpt_dir.glob("*-last.ckpt"))
    if not last_ckpts:
        raise FileNotFoundError(f"Nincs 'last.ckpt' a {ckpt_dir} mappában.")

    ckpt_path = str(last_ckpts[-1])

    return ckpt_path

### modellek betöltése

In [34]:
exp_dir = "./multi_neutral_speech"

last_ckpt_neutral = get_last_ckpt(exp_dir, "FastPitch")
checkpoint_neutral = torch.load(last_ckpt_neutral, weights_only=False) 

spec_model_neutral = FastPitchModel.load_from_checkpoint(last_ckpt_neutral, map_location="cuda")
spec_model_neutral.eval().cuda()

if not hasattr(spec_model_neutral, "speaker_emb") or spec_model_neutral.speaker_emb is None:
    print("Új speaker embedding inicializálása (10 beszélő)...")
    spec_model_neutral.speaker_emb = torch.nn.Embedding(10, 384).cuda()
    spec_model_neutral.n_speakers = 10
elif spec_model_neutral.n_speakers < 10:
    print("Frissítjük n_speakers értékét → 10")
    spec_model_neutral.n_speakers = 10

 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
I1203 21:29:38.569052 140008601651008 tokenize_and_classify.py:87] Creating ClassifyFst grammars.
[NeMo W 2025-12-03 21:30:05 en_us_arpabet:66] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2025-12-03 21:30:05 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.dataset.TTSDataset
      manifest_filepath: /home/simon/ESD/Neutral_train.json
      n_speakers: 10
      speaker_column: speaker
      sample_rate: 44100
      sup_data_path: ./MultiSpeaker/NeMoTTS_sup_data
      sup_data_types:
  

[NeMo I 2025-12-03 21:30:05 features:289] PADDING: 1
Új speaker embedding inicializálása (10 beszélő)...


In [35]:
print("Ellenőrzés:", spec_model_neutral.n_speakers, spec_model_neutral.speaker_emb.weight.shape)

Ellenőrzés: 10 torch.Size([10, 384])


In [36]:
exp_dir = "./multi_sad_speech"

last_ckpt_sad = get_last_ckpt(exp_dir, "FastPitch")
checkpoint_sad = torch.load(last_ckpt_sad, weights_only=False) 

spec_model_sad = FastPitchModel.load_from_checkpoint(last_ckpt_sad, map_location="cuda")
spec_model_sad.eval().cuda()

 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
I1203 21:30:08.791059 140008601651008 tokenize_and_classify.py:87] Creating ClassifyFst grammars.
[NeMo W 2025-12-03 21:30:37 en_us_arpabet:66] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2025-12-03 21:30:37 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.dataset.TTSDataset
      manifest_filepath: /home/simon/ESD/Sad_train.json
      n_speakers: 10
      speaker_column: speaker
      sample_rate: 44100
      sup_data_path: ./MultiSpeaker/NeMoTTS_sup_data_sad
      sup_data_types:
  

[NeMo I 2025-12-03 21:30:37 features:289] PADDING: 1


FastPitchModel(
  (mel_loss_fn): MelLoss()
  (pitch_loss_fn): PitchLoss()
  (duration_loss_fn): DurationLoss()
  (energy_loss_fn): EnergyLoss()
  (aligner): AlignmentEncoder(
    (cond_input): ConditionalInput()
    (softmax): Softmax(dim=3)
    (log_softmax): LogSoftmax(dim=3)
    (key_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(384, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (1): ReLU()
      (2): ConvNorm(
        (conv): Conv1d(768, 80, kernel_size=(1,), stride=(1,))
      )
    )
    (query_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(80, 160, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (1): ReLU()
      (2): ConvNorm(
        (conv): Conv1d(160, 80, kernel_size=(1,), stride=(1,))
      )
      (3): ReLU()
      (4): ConvNorm(
        (conv): Conv1d(80, 80, kernel_size=(1,), stride=(1,))
      )
    )
  )
  (forward_sum_loss_fn): ForwardSumLoss(
    (log_softmax): LogSoftmax(dim=-1)
    (ctc_loss): CTCLoss()


In [37]:
if not hasattr(spec_model_sad, "speaker_emb") or spec_model_sad.speaker_emb is None:
    print("Új speaker embedding inicializálása (10 beszélő)...")
    spec_model_sad.speaker_emb = torch.nn.Embedding(10, 384).cuda()
    spec_model_sad.n_speakers = 10
elif spec_model_sad.n_speakers < 10:
    print("Frissítjük n_speakers értékét → 10")
    spec_model_sad.n_speakers = 10

Új speaker embedding inicializálása (10 beszélő)...


In [38]:
exp_dir = "./multi_happy_speech"

last_ckpt_happy = get_last_ckpt(exp_dir, "FastPitch")
checkpoint_happy = torch.load(last_ckpt_happy, weights_only=False)  

spec_model_happy = FastPitchModel.load_from_checkpoint(last_ckpt_happy, map_location="cuda")
spec_model_happy.eval().cuda()

 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
I1203 21:30:41.288490 140008601651008 tokenize_and_classify.py:87] Creating ClassifyFst grammars.
[NeMo W 2025-12-03 21:31:12 en_us_arpabet:66] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2025-12-03 21:31:12 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.dataset.TTSDataset
      manifest_filepath: /home/simon/ESD/Happy_train.json
      n_speakers: 10
      speaker_column: speaker
      sample_rate: 44100
      sup_data_path: ./MultiSpeaker/NeMoTTS_sup_data_happy
      sup_data_types

[NeMo I 2025-12-03 21:31:12 features:289] PADDING: 1


FastPitchModel(
  (mel_loss_fn): MelLoss()
  (pitch_loss_fn): PitchLoss()
  (duration_loss_fn): DurationLoss()
  (energy_loss_fn): EnergyLoss()
  (aligner): AlignmentEncoder(
    (cond_input): ConditionalInput()
    (softmax): Softmax(dim=3)
    (log_softmax): LogSoftmax(dim=3)
    (key_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(384, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (1): ReLU()
      (2): ConvNorm(
        (conv): Conv1d(768, 80, kernel_size=(1,), stride=(1,))
      )
    )
    (query_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(80, 160, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (1): ReLU()
      (2): ConvNorm(
        (conv): Conv1d(160, 80, kernel_size=(1,), stride=(1,))
      )
      (3): ReLU()
      (4): ConvNorm(
        (conv): Conv1d(80, 80, kernel_size=(1,), stride=(1,))
      )
    )
  )
  (forward_sum_loss_fn): ForwardSumLoss(
    (log_softmax): LogSoftmax(dim=-1)
    (ctc_loss): CTCLoss()


In [39]:
if not hasattr(spec_model_happy, "speaker_emb") or spec_model_happy.speaker_emb is None:
    print("Új speaker embedding inicializálása (10 beszélő)...")
    spec_model_happy.speaker_emb = torch.nn.Embedding(10, 384).cuda()
    spec_model_happy.n_speakers = 10
elif spec_model_happy.n_speakers < 10:
    print("Frissítjük n_speakers értékét → 10")
    spec_model_happy.n_speakers = 10

Új speaker embedding inicializálása (10 beszélő)...


In [40]:
exp_dir = "./multi_angry_speech"

last_ckpt_angry = get_last_ckpt(exp_dir, "FastPitch")
checkpoint_angry = torch.load(last_ckpt_angry, weights_only=False) 

spec_model_angry = FastPitchModel.load_from_checkpoint(last_ckpt_angry, map_location="cuda")
spec_model_angry.eval().cuda()

 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
I1203 21:31:16.376009 140008601651008 tokenize_and_classify.py:87] Creating ClassifyFst grammars.
[NeMo W 2025-12-03 21:31:48 en_us_arpabet:66] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2025-12-03 21:31:48 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.dataset.TTSDataset
      manifest_filepath: /home/simon/ESD/Angry_train.json
      n_speakers: 10
      speaker_column: speaker
      sample_rate: 44100
      sup_data_path: ./MultiSpeaker/NeMoTTS_sup_data_angry
      sup_data_types

[NeMo I 2025-12-03 21:31:48 features:289] PADDING: 1


FastPitchModel(
  (mel_loss_fn): MelLoss()
  (pitch_loss_fn): PitchLoss()
  (duration_loss_fn): DurationLoss()
  (energy_loss_fn): EnergyLoss()
  (aligner): AlignmentEncoder(
    (cond_input): ConditionalInput()
    (softmax): Softmax(dim=3)
    (log_softmax): LogSoftmax(dim=3)
    (key_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(384, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (1): ReLU()
      (2): ConvNorm(
        (conv): Conv1d(768, 80, kernel_size=(1,), stride=(1,))
      )
    )
    (query_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(80, 160, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (1): ReLU()
      (2): ConvNorm(
        (conv): Conv1d(160, 80, kernel_size=(1,), stride=(1,))
      )
      (3): ReLU()
      (4): ConvNorm(
        (conv): Conv1d(80, 80, kernel_size=(1,), stride=(1,))
      )
    )
  )
  (forward_sum_loss_fn): ForwardSumLoss(
    (log_softmax): LogSoftmax(dim=-1)
    (ctc_loss): CTCLoss()


In [41]:
if not hasattr(spec_model_angry, "speaker_emb") or spec_model_angry.speaker_emb is None:
    print("Új speaker embedding inicializálása (10 beszélő)...")
    spec_model_angry.speaker_emb = torch.nn.Embedding(10, 384).cuda()
    spec_model_angry.n_speakers = 10
elif spec_model_angry.n_speakers < 10:
    print("Frissítjük n_speakers értékét → 10")
    spec_model_angry.n_speakers = 10

Új speaker embedding inicializálása (10 beszélő)...


In [42]:
exp_dir = "./multi_surprise_speech"

last_ckpt_surprise = get_last_ckpt(exp_dir, "FastPitch")
checkpoint_surprise = torch.load(last_ckpt_surprise, weights_only=False)

spec_model_surprise = FastPitchModel.load_from_checkpoint(last_ckpt_surprise, map_location="cuda")
spec_model_surprise.eval().cuda()

 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
I1203 21:31:52.342900 140008601651008 tokenize_and_classify.py:87] Creating ClassifyFst grammars.
[NeMo W 2025-12-03 21:32:27 en_us_arpabet:66] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2025-12-03 21:32:27 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.dataset.TTSDataset
      manifest_filepath: /home/simon/ESD/Surprise_train.json
      n_speakers: 10
      speaker_column: speaker
      sample_rate: 44100
      sup_data_path: ./MultiSpeaker/NeMoTTS_sup_data_surprise
      sup_data

[NeMo I 2025-12-03 21:32:27 features:289] PADDING: 1


FastPitchModel(
  (mel_loss_fn): MelLoss()
  (pitch_loss_fn): PitchLoss()
  (duration_loss_fn): DurationLoss()
  (energy_loss_fn): EnergyLoss()
  (aligner): AlignmentEncoder(
    (cond_input): ConditionalInput()
    (softmax): Softmax(dim=3)
    (log_softmax): LogSoftmax(dim=3)
    (key_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(384, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (1): ReLU()
      (2): ConvNorm(
        (conv): Conv1d(768, 80, kernel_size=(1,), stride=(1,))
      )
    )
    (query_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(80, 160, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (1): ReLU()
      (2): ConvNorm(
        (conv): Conv1d(160, 80, kernel_size=(1,), stride=(1,))
      )
      (3): ReLU()
      (4): ConvNorm(
        (conv): Conv1d(80, 80, kernel_size=(1,), stride=(1,))
      )
    )
  )
  (forward_sum_loss_fn): ForwardSumLoss(
    (log_softmax): LogSoftmax(dim=-1)
    (ctc_loss): CTCLoss()


In [43]:
if not hasattr(spec_model_surprise, "speaker_emb") or spec_model_surprise.speaker_emb is None:
    print("Új speaker embedding inicializálása (10 beszélő)...")
    spec_model_surprise.speaker_emb = torch.nn.Embedding(10, 384).cuda()
    spec_model_surprise.n_speakers = 10
elif spec_model_surprise.n_speakers < 10:
    print("Frissítjük n_speakers értékét → 10")
    spec_model_surprise.n_speakers = 10

Új speaker embedding inicializálása (10 beszélő)...


## gpt felolvastatás

In [ ]:
client = OpenAI(api_key="API_KEY")

In [46]:

text_data = load_pdf("./GPT/Everything_about_VIK.pdf")

chunks = smart_chunk_text(text_data)
print("Chunkok száma:", len(chunks))

index, embeddings, text_data = create_embeddings(chunks)
print("Embedding és index létrehozva.")


Chunkok száma: 248
Embedding és index létrehozva.


In [48]:
speakers = [
    "Default",
    "1 - Male",
    "2 - Male",
    "3 - Male",
    "4 - Male",
    "5 - Female",
    "6 - Female",
    "7 - Female",
    "8 - Female",
    "9 - Female",
    "10 - Male"
]
messages=[]

In [49]:
import librosa

def estimate_speech_speed(audio_path, text):
   
    duration_sec = librosa.get_duration(path=audio_path)
    duration_min = duration_sec / 60
    
    words = text.strip().split()
    num_words = len(words)

    if duration_min > 0:
        wpm = num_words / duration_min
    else:
        wpm = 0

    if wpm < 50:
        speed_label = "slow"
    elif wpm < 140:
        speed_label = "normal"
    else:
        speed_label = "fast"

    return wpm, speed_label, num_words, duration_sec


In [70]:
def chat_and_tts(user_input, speaker_choice,pred_emo,gender,speed_label):
    global messages
    
    print(f"Input: '{user_input}'")  
    print(f"Speaker: {speaker_choice}, Emotion: {pred_emo}, Gender: {gender}")  
    
    if not user_input.strip():
        return "Please enter a question.", None, "neutral"
   
    context = retrieve_context(user_input, index, text_data, top_k=8)
    system_prompt = {
        "role": "system",
        "content":
        "You are an empathetic, emotionally adaptive AI assistant designed specifically for helping first-year "
        "students at the Budapest University of Technology and Economics (BME). Always respond in English. "
        "You should assume that every question relates to university life, BME studies, courses, schedules, "
        "student responsibilities, or general academic difficulties—unless the user clearly indicates otherwise. "
        "Your primary goal is to support the student emotionally while also giving accurate, context-based answers. "
        "Always check the provided RAG context first. If the context contains information relevant to the question, "
        "you MUST rely only on that information. If the context does NOT contain relevant information, give a short, "
        "general, helpful answer, but NEVER hallucinate facts about BME, courses, requirements, deadlines, or policies. "
        "If something is unknown, say it kindly and offer general academic guidance instead. "
        "Your answers must be concise: maximum 3 short, clear sentences. "
        "Never mention the context directly. Never explain your reasoning process. "
        "Emotion handling rules: "
        "- Neutral user: stay calm, slightly positive, and supportive. "
        "- Happy user: lightly mirror their positivity. "
        "- Sad user: be warm and empathetic but not overly emotional. "
        "- Surprised user: respond calmly and confidently. "
        "- Angry user: do NOT mirror anger; stay steady, understanding, and solution-oriented. "
        "Audio-based emotion predictions are uncertain and should only influence your tone softly. "
        "Output format requirement: always respond strictly as a JSON object: "
        "{\"text\": <your answer>, \"emotion\": <one of [happy, sad, angry, neutral, surprise]>}. "
        "Do not include anything outside this JSON. "
        f"RAG context (may or may not contain relevant information): {context}\n"
        f"Predicted user emotion from audio (uncertain): {pred_emo}"
    }

    
    conversation_input = [
        system_prompt,
        *messages,
        {"role": "user", "content": user_input}
    ]


    print(datetime.now())
   
    chat = client.chat.completions.create(
        model="gpt-5-nano",
        messages=conversation_input
    )
    
    reply = chat.choices[0].message.content.strip()
    print(datetime.now())
    print(f"GPT válasz: '{reply}'")  
    
    messages.append({"role": "user", "content": user_input})
    messages.append({"role": "assistant", "content": reply})
   
    MAX_PAIRS = 15

    if len(messages) > MAX_PAIRS * 2:
        messages = messages[-MAX_PAIRS*2:]

    
    if speaker_choice=="Default":
        if gender=="Male":
            speaker_id=6
        else:
            speaker_id=1
    else:    
        speaker_id = int(speaker_choice.split(" ")[0]) - 1

        
    print(f"Speaker ID: {speaker_id}")  

    try:
        data = json.loads(reply)
        text = data["text"]
        
        emotion = data["emotion"].lower()
    except:
        print(f"JSON hiba: {e}")  
        return "JSON parse error", None, "neutral"

    model_map = {
        "happy": spec_model_happy,
        "sad": spec_model_sad,
        "angry": spec_model_angry,
        "neutral": spec_model_neutral,
        "surprise": spec_model_surprise
    }
    
    if speed_label == "slow":
        speed = 0.8
    elif speed_label == "fast":
        speed = 1.2
    else:
        speed = 1.0


    selected = model_map.get(emotion, spec_model_neutral)

    
    
    spec, audio = infer_multispeaker(
        spec_gen_model=selected,
        vocoder_model=vocoder,
        text=text,
        speaker_id=speaker_id,
        pace=speed
    )


    audio_np = np.array(audio, dtype=np.float32)

    output_path=f"output_{int(time.time() * 1000)}.wav"
    print(datetime.now())
    sf.write(output_path, audio_np.T, 44100)
    print(f" Hang mentve: {output_path}") 

    
    return text, output_path, emotion 

def chat_and_tts_from_speech(audio_input, speaker_choice):
    if audio_input is None:
        return "No audio detected.", None, "neutral"

    
    samplerate, data = audio_input
    wav_path = "record.wav"
    write(wav_path, samplerate, data.astype(np.int16))


    result = whisper(wav_path)
    recognized_text = result["text"]
    
    print(f" Whisper felismerte: '{recognized_text}'")
    
    noise_words = {
        "you", "yo", "yeah", "ya", "okay", "ok", "okey", "alright",
        "thank you", "thanks","ha", "ah", "uh", "um", "hmm", "mmm", "uhh", "eh", "oh", "huh",
        "mmmhm", "mmhmm", "mhmm","m", "mm", "mmm", "h", "uhm", "erm","hey", "hi", "yo.", "huh.", "oh.", "so", "well", "right",
        "oh okay", "oh ok", "i see", "hmm okay","", " ", ".", ",", "?", "!", "-", "_"
    }
    normalized_text = recognized_text.lower().strip()

    
    normalized_text = normalized_text.translate(str.maketrans("", "", string.punctuation))

    normalized_text = " ".join(normalized_text.split())



    if normalized_text in noise_words:
        print("Noise word detected → ignored.")
        return "", None, "neutral", recognized_text
    
    wpm, speed_label, num_words, duration_sec = estimate_speech_speed(wav_path, recognized_text)
    print(f" Beszédtempó: {wpm:.1f} WPM ({speed_label}) | {num_words} szó, {duration_sec:.2f} mp")
    print(datetime.now())
    
    audio_path=wav_path
    predicted_emotion = predict_emotion(audio_path, model, enc)
    print(f" Érzelem: {predicted_emotion}")
    print(datetime.now())
    
   
    gender = estimate_gender(audio_path)

    print(f"Nem: {gender}")
    print(datetime.now())
    text_reply, audio_output_path, emotion = chat_and_tts(recognized_text, speaker_choice,predicted_emotion,gender,speed_label)
    print(f"Válasz: '{text_reply}'")  
    print(f"Hangfájl: {audio_output_path}")  
    print(f"Kimenet érzelem: {emotion}")  
    
    return text_reply, audio_output_path, emotion, recognized_text



In [51]:
torch.cuda.empty_cache()

In [79]:
import gradio as gr
import numpy as np
import os
import time
import threading
import queue
import tempfile
import soundfile as sf


SAMPLE_RATE = 16000
audio_buffer = np.array([], dtype=np.float32)
last_speech_time = 0
is_recording = False

MIN_SPEECH_DURATION = 1.0
SILENCE_THRESHOLD = 0.030
SILENCE_DURATION = 1.5
MIN_CHUNK_LENGTH = 2000
MIN_RMS_FOR_SPEECH = 0.028
MIN_ZCR_FOR_SPEECH = 0.009
MIN_CONTINUOUS_SPEECH = 0.15


task_queue = queue.Queue()
audio_results = queue.Queue()

def background_processor():
    while True:
        sample_rate, audio_int16, speaker_choice = task_queue.get()

        try:
            text_reply, audio_path, emotion, recognized_text = chat_and_tts_from_speech(
                (sample_rate, audio_int16),
                speaker_choice
            )
            if audio_path and os.path.exists(audio_path):
                audio_data, sr = sf.read(audio_path, dtype="float32")
                if audio_data.ndim > 1:
                    audio_data = np.mean(audio_data, axis=1)

                audio_results.put((sr, audio_data))

        except Exception as e:
            print("Background error:", e)

        task_queue.task_done()

threading.Thread(target=background_processor, daemon=True).start()


def handle_stream(audio_chunk, speaker_choice):
    global audio_buffer, last_speech_time, is_recording

    if audio_chunk is None:
        return gr.update()

    try:
        if isinstance(audio_chunk, tuple):
            sample_rate, audio_data = audio_chunk
        else:
            audio_data = audio_chunk
            sample_rate = SAMPLE_RATE

        if audio_data.ndim > 1:
            audio_data = np.mean(audio_data, axis=1)

        audio_data = audio_data.astype(np.float32)
        if np.max(np.abs(audio_data)) > 1:
            audio_data /= 32768.0

        energy = np.sqrt(np.mean(audio_data ** 2))
        zcr = np.mean(np.abs(np.diff(np.sign(audio_data))))
        chunk_dur = len(audio_data) / sample_rate
        valid = len(audio_data) >= MIN_CHUNK_LENGTH

        has_speech = (
            energy > SILENCE_THRESHOLD
            and energy > MIN_RMS_FOR_SPEECH
            and zcr > MIN_ZCR_FOR_SPEECH
            and valid
            and chunk_dur >= MIN_CONTINUOUS_SPEECH
        )

        now = time.time()

        if has_speech and not is_recording:
            is_recording = True
            audio_buffer = audio_data.copy()
            last_speech_time = now
            return gr.update()

        if is_recording:
            audio_buffer = np.concatenate([audio_buffer, audio_data])
            if has_speech:
                last_speech_time = now

        if is_recording:
            silence = now - last_speech_time
            duration = len(audio_buffer) / sample_rate

            if silence >= SILENCE_DURATION and duration >= MIN_SPEECH_DURATION:
                audio_int16 = (audio_buffer * 32767).astype(np.int16)
                task_queue.put((sample_rate, audio_int16, speaker_choice))

                audio_buffer = np.array([], dtype=np.float32)
                is_recording = False
                last_speech_time = 0

                return gr.update()

        return gr.update()

    except Exception as e:
        print("Stream error:", e)
        return gr.update()


def poll_audio():
    if audio_results.empty():
        return gr.update()

    sr, audio = audio_results.get()

    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    sf.write(tmp.name, audio, sr)

    return tmp.name


with gr.Blocks(
    theme="soft",
    css="""
        /* Hide ONLY the output audio player UI, but keep it functional */
        .output-audio-box audio {
            display: none !important;
        }
        .output-audio-box {
            height: 0px !important;
            padding: 0 !important;
            margin: 0 !important;
            overflow: hidden !important;
        }
    """
) as demo:

    gr.Markdown("## 🎙️ Emotional AI Assistant")

    with gr.Row():

        with gr.Column(scale=1):
            audio_input = gr.Audio(type="numpy", streaming=True, label="Microphone")

            speaker_drop = gr.Dropdown(
                choices=speakers,
                value=speakers[0],
                label="Voice"
            )
            voice_state = gr.State(speakers[0])

            speaker_drop.change(lambda x: x, inputs=speaker_drop, outputs=voice_state)

        with gr.Column(scale=2):
           
            audio_output = gr.Audio(
                type="filepath",
                autoplay=True,
                label="",
                elem_classes=["output-audio-box"]
            )

    audio_input.stream(
        fn=handle_stream,
        inputs=[audio_input, voice_state],
        outputs=[audio_output]
    )

    audio_input.change(
        fn=poll_audio,
        inputs=None,
        outputs=audio_output
    )


demo.launch(server_name="0.0.0.0", server_port=8638, share=True)


* Running on local URL:  http://0.0.0.0:8638
* Running on public URL: https://2f10a467a527bba33c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [67]:
import string